## Project 3: RETAIL STORE RETURN POLICY CHANGE 




In [2]:
import pandas as pd
from pandas import DataFrame
import numpy as np
from pandasql import sqldf
import re

### Importing Data

In [3]:
trans = pd.read_csv('/Users/mugdhapotdar/Downloads/Transactions and customer attributes.txt', sep = '\t')

In [4]:
trans.head(3)

,customer_id,purchase_date,transaction_id,store_number,net_purchase_amount,sku,sales_assoc_1,return,return_date,return_store,...,est_income_code,ethnic_code,homeowner_code,length_of_residence,child,year,month,month_index,summary,Brand_number
0,1000000830585,18DEC2013:00:00:00,5184137,2,58.490002,19950612,0.0,0,NaN,NaN,...,6.0,N,O,10.0,Y,2013,DEC,53,NaN,1
1,35931485,15DEC2013:00:00:00,5121239,2,179.990010,19117175,0.0,0,NaN,NaN,...,7.0,N,O,15.0,N,2013,DEC,53,4.0,1
2,36792453,05JUN2013:00:00:00,4609592,2,136.929990,19205426,0.0,0,NaN,NaN,...,2.0,N,O,8.0,Y,2013,JUN,47,21.0,1


In [5]:
trans.columns

Index([u'customer_id', u'purchase_date', u'transaction_id', u'store_number',
       u'net_purchase_amount', u'sku', u'sales_assoc_1', u'return',
       u'return_date', u'return_store', u'time_to_return', u'gender',
       u'age_band', u'est_income_code', u'ethnic_code', u'homeowner_code',
       u'length_of_residence', u'child', u'year', u'month', u'month_index',
       u'summary', u'Brand_number'],
      dtype='object')

In [6]:
store = pd.read_csv('/Users/mugdhapotdar/Downloads/store_attributes.csv')

In [7]:
store.columns

Index([u'store_number', u'month_index', u'month', u'year',
       u'store_average_price', u'store_number_of_skus', u'sa_gender',
       u'sa_full_time', u'sa_avg_years_of_exp', u'sa_married',
       u'sa_avg_rate_of_pay', u'sa_dependent', u'Brand_number',
       u'sales_volume_group'],
      dtype='object')

### SQLDF to run SQL queries on Pandas Dataframes

In [8]:
# SQLDF to run SQL queries on Pandas DataFrames
# defining a function to specify second parameter for sqldf function as global()
py_sql = (lambda s: sqldf(s, globals()))

In [9]:
# For Returns (Q2, Q4 and Q6) considering only those rows where return = 1 this is for Q3 and Q4
trans_p = py_sql("SELECT * FROM trans where return = 1;")

In [10]:
trans_p.head(2)

,customer_id,purchase_date,transaction_id,store_number,net_purchase_amount,sku,sales_assoc_1,return,return_date,return_store,...,est_income_code,ethnic_code,homeowner_code,length_of_residence,child,year,month,month_index,summary,Brand_number
0,100760.0,20DEC2013:00:00:00,5210109.0,2.0,72.239998,17934779.0,0.0,1.0,08JAN2014:00:00:00,2.0,...,6.0,Z,O,5.0,Y,2013.0,DEC,53.0,4.0,1.0
1,38198535.0,23JUL2013:00:00:00,4687325.0,2.0,369.989990,18669945.0,0.0,1.0,30AUG2013:00:00:00,2.0,...,1.0,N,R,0.0,N,2013.0,JUL,48.0,1.0,1.0


### Question 1. What is the impact of the policy change on online channel sales? 
                DataFrame q1_final

In [11]:
# q1 part 1
q1 = py_sql("SELECT store_number, month, Brand_number, \
                sum(case when gender in ('F') then 1 else 0 end) as Female, \
                sum(case when gender in ('M','F') then 1 else 0 end) as Total_Female, \
                avg(age_band) as Age_Band_Ratio, \
                sum(case when homeowner_code = 'O' then 1 else 0 end) as HomeOwner, \
                sum(case when homeowner_code in ('O','R') then 1 else 0 end) as Total_Homeowner, \
                sum(case when child = 'Y' then 1 else 0 end) as Child, \
                sum(case when child in ('Y','N') then 1 else 0 end) as Total_Child, \
                avg(est_income_code) as Income_Ratio, \
                sum(net_purchase_amount) as Sales_Price, \
                count(*) as Sales_Quantity, \
                (case when month_index > 50 then 1 else 0 end) as Policy_Change  \
                FROM trans \
                GROUP BY store_number, month ;")

In [12]:
q1.head(2)

,store_number,month,Brand_number,Female,Total_Female,Age_Band_Ratio,HomeOwner,Total_Homeowner,Child,Total_Child,Income_Ratio,Sales_Price,Sales_Quantity,Policy_Change
0,2.0,APR,1.0,19153,35919,4.472780,27411,43293,18053,43293,5.242857,6.620615e+06,43417,0
1,2.0,AUG,1.0,12189,22895,4.045565,19186,28998,12302,28998,5.407407,5.688315e+06,29338,0


In [13]:
# q1 part 2
# online store with all ratios 
q11 = py_sql("SELECT store_number, month, \
                 Age_Band_Ratio, \
                Female / Total_Female as Gender_Ratio, \
                HomeOwner / Total_Homeowner as HomeOwner_Ratio, \
                Child / Total_Child as Child_Ratio, \
                Income_Ratio, \
                Sales_Price, \
                Sales_Quantity, \
                Policy_Change,  \
               (case when store_number in (2,6) then 1 else 0 end ) as Treatment \
                FROM q1 \
                WHERE store_number = 2 or store_number = 6 or store_number = 10 \
                GROUP BY store_number, month;")

In [14]:
q11.head(2)

,store_number,month,Age_Band_Ratio,Gender_Ratio,HomeOwner_Ratio,Child_Ratio,Income_Ratio,Sales_Price,Sales_Quantity,Policy_Change,Treatment
0,2.0,APR,4.472780,0.533228,0.633151,0.416996,5.242857,6.620615e+06,43417.0,0.0,1
1,2.0,AUG,4.045565,0.532387,0.661632,0.424236,5.407407,5.688315e+06,29338.0,0.0,1


In [15]:
# q1 final Data Frame
q1_final = py_sql("SELECT td.*, st.store_number_of_skus \
                   FROM q11 td \
                   INNER JOIN store st \
                   ON td.store_number = st.store_number \
                   AND td.month = st.month;")
            

In [16]:
q1_final.head(2)

,store_number,month,Age_Band_Ratio,Gender_Ratio,HomeOwner_Ratio,Child_Ratio,Income_Ratio,Sales_Price,Sales_Quantity,Policy_Change,Treatment,store_number_of_skus
0,2.0,APR,4.472780,0.533228,0.633151,0.416996,5.242857,6.620615e+06,43417.0,0.0,1.0,11134.0
1,2.0,AUG,4.045565,0.532387,0.661632,0.424236,5.407407,5.688315e+06,29338.0,0.0,1.0,10352.0


In [17]:
q1_final.to_csv("M_q1.csv")

### Q.2 What is the impact of the policy change on physical store sales?
         Dataframe q2_final

In [18]:
q21 = py_sql("SELECT store_number, month, \
                     Age_Band_Ratio, \
                     Female / Total_Female as Gender_Ratio, \
                     HomeOwner / Total_Homeowner as HomeOwner_Ratio, \
                     Child / Total_Child as Child_Ratio, \
                     Income_Ratio, \
                     Sales_Price, \
                     Sales_Quantity, \
                     Policy_Change,  \
                     (case when Brand_number in (2,3,5) then 1 else 0 end ) as Treatment \
              FROM q1 \
              WHERE store_number not in (2,6,10) \
              GROUP BY store_number, month;")

In [19]:
q21.head(2)

,store_number,month,Age_Band_Ratio,Gender_Ratio,HomeOwner_Ratio,Child_Ratio,Income_Ratio,Sales_Price,Sales_Quantity,Policy_Change,Treatment
0,15.0,APR,4.434783,0.866667,0.695652,0.434783,4.173913,7398.890057,335.0,0.0,0
1,15.0,AUG,5.200000,0.866667,0.733333,0.800000,5.200000,11418.600079,526.0,0.0,0


In [20]:
# q2 final Data Frame
q2_final = py_sql("SELECT td1.*, st1.store_average_price, st1.sa_gender, st1.sa_full_time, \
                          st1.sa_avg_years_of_exp, st1.sa_married, st1.sa_avg_rate_of_pay, st1.sa_dependent, \
                          st1.store_number_of_skus, st1.Brand_number, st1.sales_volume_group \
                   FROM q21 td1 \
                   INNER JOIN store st1 \
                   ON td1.store_number = st1.store_number \
                   AND td1.month = st1.month;")

In [21]:
q2_final.head(2)

,store_number,month,Age_Band_Ratio,Gender_Ratio,HomeOwner_Ratio,Child_Ratio,Income_Ratio,Sales_Price,Sales_Quantity,Policy_Change,...,store_average_price,sa_gender,sa_full_time,sa_avg_years_of_exp,sa_married,sa_avg_rate_of_pay,sa_dependent,store_number_of_skus,Brand_number,sales_volume_group
0,16.0,APR,5.607143,0.619048,0.714286,0.678571,6.107143,6742.280023,327.0,0.0,...,32.91,1.0,0.67,1.0,0.0,10.67,0.00,138.0,8.0,1.0
1,16.0,AUG,5.769231,0.890625,0.602941,0.500000,6.411765,12399.640039,640.0,0.0,...,31.24,1.0,1.00,7.4,0.0,10.69,0.29,216.0,8.0,1.0


In [22]:
q2_final.to_csv("M_q2.csv")

### Question 3. What is the impact of the policy change on online channel returns? 
                 DataFrame q3_final

In [23]:
# Q3 part 1
q3 = py_sql("SELECT store_number, month,\
                sum(case when gender in ('F') then 1 else 0 end) as Female, \
                sum(case when gender in ('M','F') then 1 else 0 end) as Total_Female, \
                avg(age_band) as Age_Band_Ratio, \
                sum(case when homeowner_code = 'O' then 1 else 0 end) as HomeOwner, \
                sum(case when homeowner_code in ('O','R') then 1 else 0 end) as Total_Homeowner, \
                sum(case when child = 'Y' then 1 else 0 end) as Child, \
                sum(case when child in ('Y','N') then 1 else 0 end) as Total_Child, \
                avg(est_income_code) as Income_Ratio, \
                sum(net_purchase_amount) as return_amount, \
                count(*) as return_quantity, \
                (case when month_index > 50 then 1 else 0 end) as Policy_Change  \
                FROM trans_p \
                GROUP BY store_number, month ;")

In [24]:
q3.head(2)

,store_number,month,Female,Total_Female,Age_Band_Ratio,HomeOwner,Total_Homeowner,Child,Total_Child,Income_Ratio,return_amount,return_quantity,Policy_Change
0,2.0,APR,1927,3096,4.257434,2360,3750,1547,3750,5.332000,1.090199e+06,3768,0
1,2.0,AUG,1309,2390,3.049206,2061,3166,1294,3166,5.500316,9.765532e+05,3211,0


In [25]:
# Q3 part 2
q31 = py_sql("SELECT store_number, month, \
                 Age_Band_Ratio, \
                Female / Total_Female as Gender_Ratio, \
                HomeOwner / Total_Homeowner as HomeOwner_Ratio, \
                Child / Total_Child as Child_Ratio, \
                Income_Ratio, \
                return_amount, \
                return_quantity, \
                Policy_Change,  \
               (case when store_number in (2,6) then 1 else 0 end ) as Treatment \
                FROM q3 \
                WHERE store_number = 2 or store_number = 6 or store_number = 10 \
                GROUP BY store_number, month;")

In [26]:
q31.head(2)

,store_number,month,Age_Band_Ratio,Gender_Ratio,HomeOwner_Ratio,Child_Ratio,Income_Ratio,return_amount,return_quantity,Policy_Change,Treatment
0,2.0,APR,4.257434,0.622416,0.629333,0.412533,5.332000,1.090199e+06,3768.0,0.0,1
1,2.0,AUG,3.049206,0.547699,0.650979,0.408718,5.500316,9.765532e+05,3211.0,0.0,1


In [27]:
q31['Sales_Price'] = q11['Sales_Price']
q31['Sales_Quantity'] = q11['Sales_Quantity']

In [28]:
q31.head(2)

,store_number,month,Age_Band_Ratio,Gender_Ratio,HomeOwner_Ratio,Child_Ratio,Income_Ratio,return_amount,return_quantity,Policy_Change,Treatment,Sales_Price,Sales_Quantity
0,2.0,APR,4.257434,0.622416,0.629333,0.412533,5.332000,1.090199e+06,3768.0,0.0,1,6.620615e+06,43417.0
1,2.0,AUG,3.049206,0.547699,0.650979,0.408718,5.500316,9.765532e+05,3211.0,0.0,1,5.688315e+06,29338.0


In [29]:
#Q3 Final
q3_final = py_sql("SELECT td.*, st.store_number_of_skus \
                   FROM q31 td \
                   INNER JOIN store st \
                   ON td.store_number = st.store_number \
                   AND td.month = st.month;")

In [30]:
q3_final.to_csv('M_q3.csv')

### Q.4 What is the impact of the policy change on physical store returns? 

In [31]:
# Q4 part 1
q4 = py_sql("SELECT store_number, month, Brand_number, \
                sum(case when gender in ('F') then 1 else 0 end) as Female, \
                sum(case when gender in ('M','F') then 1 else 0 end) as Total_Female, \
                avg(age_band) as Age_Band_Ratio, \
                sum(case when homeowner_code = 'O' then 1 else 0 end) as HomeOwner, \
                sum(case when homeowner_code in ('O','R') then 1 else 0 end) as Total_Homeowner, \
                sum(case when child = 'Y' then 1 else 0 end) as Child, \
                sum(case when child in ('Y','N') then 1 else 0 end) as Total_Child, \
                avg(est_income_code) as Income_Ratio, \
                sum(net_purchase_amount) as return_amount, \
                count(*) as return_quantity, \
                (case when month_index > 50 then 1 else 0 end) as Policy_Change  \
                FROM trans_p \
                GROUP BY store_number, month ;")

In [32]:
q4.head(2)

,store_number,month,Brand_number,Female,Total_Female,Age_Band_Ratio,HomeOwner,Total_Homeowner,Child,Total_Child,Income_Ratio,return_amount,return_quantity,Policy_Change
0,2.0,APR,1.0,1927,3096,4.257434,2360,3750,1547,3750,5.332000,1.090199e+06,3768,0
1,2.0,AUG,1.0,1309,2390,3.049206,2061,3166,1294,3166,5.500316,9.765532e+05,3211,0


In [33]:
# Q4 part 2
q41 = py_sql("SELECT store_number, month, Brand_number \
                     Age_Band_Ratio, \
                     Female / Total_Female as Gender_Ratio, \
                     HomeOwner / Total_Homeowner as HomeOwner_Ratio, \
                     Child / Total_Child as Child_Ratio, \
                     Income_Ratio, \
                     return_amount, \
                     return_quantity, \
                     Policy_Change,  \
                     (case when Brand_number in (2,3,5) then 1 else 0 end ) as Treatment \
              FROM q4 \
              WHERE store_number not in (2,6,10) \
              GROUP BY store_number, month;")

In [34]:
q41['Sales_Price'] = q21['Sales_Price']
q41['Sales_Quantity'] = q21['Sales_Quantity']

In [35]:
q41.head(2)

,store_number,month,Age_Band_Ratio,Gender_Ratio,HomeOwner_Ratio,Child_Ratio,Income_Ratio,return_amount,return_quantity,Policy_Change,Treatment,Sales_Price,Sales_Quantity
0,15.0,APR,8.0,1.0,0.666667,0.333333,2.666667,1172.440036,26.0,0.0,0,7398.890057,335.0
1,15.0,AUG,8.0,NaN,NaN,NaN,NaN,2338.850041,44.0,0.0,0,11418.600079,526.0


In [36]:
#Q4 Final
q4_final = py_sql("SELECT td.*, st.store_average_price, st.sa_gender, st.sa_full_time, \
                          st.sa_avg_years_of_exp, st.sa_married, st.sa_avg_rate_of_pay, st.sa_dependent, \
                          st.store_number_of_skus, st.Brand_number, st.sales_volume_group \
                   FROM q41 td \
                   INNER JOIN store st \
                   ON td.store_number = st.store_number \
                   AND td.month = st.month;")

In [37]:
q4_final.head(2)

,store_number,month,Age_Band_Ratio,Gender_Ratio,HomeOwner_Ratio,Child_Ratio,Income_Ratio,return_amount,return_quantity,Policy_Change,...,store_average_price,sa_gender,sa_full_time,sa_avg_years_of_exp,sa_married,sa_avg_rate_of_pay,sa_dependent,store_number_of_skus,Brand_number,sales_volume_group
0,16.0,APR,8.0,0.375,0.875,0.500,7.250,789.940009,30.0,0.0,...,32.91,1.0,0.67,1.0,0.0,10.67,0.00,138.0,8.0,1.0
1,16.0,AUG,8.0,1.000,0.250,0.125,7.125,1101.579995,35.0,0.0,...,31.24,1.0,1.00,7.4,0.0,10.69,0.29,216.0,8.0,1.0


In [38]:
q4_final.columns

Index([u'store_number', u'month', u'Age_Band_Ratio', u'Gender_Ratio',
       u'HomeOwner_Ratio', u'Child_Ratio', u'Income_Ratio', u'return_amount',
       u'return_quantity', u'Policy_Change', u'Treatment', u'Sales_Price',
       u'Sales_Quantity', u'store_average_price', u'sa_gender',
       u'sa_full_time', u'sa_avg_years_of_exp', u'sa_married',
       u'sa_avg_rate_of_pay', u'sa_dependent', u'store_number_of_skus',
       u'Brand_number', u'sales_volume_group'],
      dtype='object')

In [39]:
q4_final.to_csv('M_q4.csv')

### Question 5. What is the impact of the policy change on customers purchase behavior?
#### Two DataFrames one for Online Store q5_final_online and other for Physical Store

### ONLINE STORE

In [40]:
q5_online = py_sql("SELECT * from trans where store_number in (2,6,10)")

In [41]:
# Data Cleansing
q5_online['customer_id2'] = q5_online.customer_id.astype(int).astype(str)
q5_online['store_number2'] = q5_online.store_number.astype(int).astype(str)

In [42]:
q5_online['customer_id'].nunique()

399960

In [43]:
q5_online_treat = py_sql("SELECT *, \
                                (case when month_index > 50 then 1 else 0 end) as Policy_Change, \
                                (case when store_number2 in ('2','6') then 1 else 0 end) as Treatment \
                      FROM q5_online \
                      WHERE store_number2 in ('2','6') \
                      AND customer_id2 in \
                      (                    \
                         SELECT DISTINCT customer_id2 \
                         FROM q5_online \
                         WHERE month_index > 50 \
                         AND store_number2 in ('2','6') \
                         AND customer_id2 in \
                         (                  \
                            SELECT DISTINCT customer_id2  \
                            FROM q5_online \
                            where month_index <= 50 \
                            AND store_number2 in ('2','6')  \
                         ) \
                       )\
                       AND customer_id2 not in \
                       (                  \
                           SELECT DISTINCT customer_id2 \
                           FROM q5_online \
                           WHERE month_index > 50 \
                           AND store_number2 in ('10') \
                           AND customer_id2 in \
                           (                  \
                               SELECT DISTINCT customer_id2  \
                               FROM q5_online \
                               WHERE month_index <= 50 \
                               AND store_number2 in ('10')  \
                           )\
                        ) \
                        ;" )

In [44]:
#Data Cleansing
q5_online_treat['Treatment'] = q5_online_treat.Treatment.astype(int).astype(str)
q5_online_treat['Policy_Change'] = q5_online_treat.Policy_Change.astype(int).astype(str)

In [45]:
#Checking if the sub table has same number of customer ids
q5_online_treat['customer_id2'].nunique()

10374

In [46]:
q5_online_treat['Treatment'].unique()

array(['1'], dtype=object)

In [47]:
q5_online_treat.head(2)

,customer_id,purchase_date,transaction_id,store_number,net_purchase_amount,sku,sales_assoc_1,return,return_date,return_store,...,child,year,month,month_index,summary,Brand_number,customer_id2,store_number2,Policy_Change,Treatment
0,4864145.0,10DEC2013:00:00:00,5051471.0,2.0,35.00,19043892.0,0.0,0.0,None,NaN,...,Y,2013.0,DEC,53.0,9.0,1.0,4864145,2,1,1
1,36904789.0,20JAN2014:00:00:00,5331636.0,2.0,18.75,18982363.0,0.0,0.0,None,NaN,...,Y,2014.0,JAN,54.0,9.0,1.0,36904789,2,1,1


In [48]:
q5_online_cont = py_sql("SELECT *, \
                                (case when month_index > 50 then 1 else 0 end) as Policy_Change, \
                                (case when store_number2 in ('2','6') then 1 else 0 end) as Treatment \
                      FROM q5_online \
                      WHERE store_number2  in ('10') \
                      AND customer_id2 in \
                      (                    \
                         SELECT DISTINCT customer_id2 \
                         FROM q5_online \
                         WHERE month_index > 50 \
                         AND store_number2 in ('10') \
                         AND customer_id2 in \
                         (                  \
                            SELECT DISTINCT customer_id2  \
                            FROM q5_online \
                            where month_index <= 50 \
                            AND store_number2 in ('10')  \
                         ) \
                       )\
                       AND customer_id2 not in \
                       (                  \
                          SELECT DISTINCT customer_id2 \
                          FROM q5_online \
                          WHERE month_index > 50 \
                          AND store_number2 in ('2','6') \
                          AND customer_id2 in \
                          (                  \
                             SELECT DISTINCT customer_id2  \
                             FROM q5_online \
                             WHERE month_index <= 50 \
                             AND store_number2 in ('2','6')  \
                          )\
                        ) \
                        ;" )

In [49]:
q5_online_cont['customer_id2'].nunique()

966

In [50]:
#Data Cleansing
q5_online_cont['Treatment'] = q5_online_cont.Treatment.astype(int).astype(str)
q5_online_cont['Policy_Change'] = q5_online_cont.Policy_Change.astype(int).astype(str)

In [51]:
q5_online_cont.head(2)

,customer_id,purchase_date,transaction_id,store_number,net_purchase_amount,sku,sales_assoc_1,return,return_date,return_store,...,child,year,month,month_index,summary,Brand_number,customer_id2,store_number2,Policy_Change,Treatment
0,37841418.0,04JUN2013:00:00:00,4607226.0,10.0,5.99,17613712.0,0.0,0.0,None,NaN,...,None,2013.0,JUN,47.0,16.0,1.0,37841418,10,0,0
1,31619161.0,26SEP2013:00:00:00,4797609.0,10.0,11.49,18590133.0,0.0,0.0,None,NaN,...,None,2013.0,SEP,50.0,21.0,1.0,31619161,10,0,0


In [52]:
q5_online_cont['customer_id2'].nunique()

966

In [53]:
q5_online_treat['customer_id2'].nunique()

10374

In [54]:
#Creating new dataframe for Q6 without aggregation with both Treatment and Control
q5_without_aggregation_onl = pd.concat([q5_online_treat, q5_online_cont])

In [55]:
q5_without_aggregation_onl.count()

customer_id            47564
purchase_date          47564
transaction_id         47564
store_number           47564
net_purchase_amount    47564
sku                    47564
sales_assoc_1          47525
return                 47564
return_date             5195
return_store            5195
time_to_return          5195
gender                 41685
age_band               43307
est_income_code        43590
ethnic_code            43669
homeowner_code         43590
length_of_residence    43590
child                  43590
year                   47564
month                  47564
month_index            47564
summary                46179
Brand_number           47564
customer_id2           47564
store_number2          47564
Policy_Change          47564
Treatment              47564
dtype: int64

In [56]:
q5_without_aggregation_onl.head(2)

,customer_id,purchase_date,transaction_id,store_number,net_purchase_amount,sku,sales_assoc_1,return,return_date,return_store,...,child,year,month,month_index,summary,Brand_number,customer_id2,store_number2,Policy_Change,Treatment
0,4864145.0,10DEC2013:00:00:00,5051471.0,2.0,35.00,19043892.0,0.0,0.0,None,NaN,...,Y,2013.0,DEC,53.0,9.0,1.0,4864145,2,1,1
1,36904789.0,20JAN2014:00:00:00,5331636.0,2.0,18.75,18982363.0,0.0,0.0,None,NaN,...,Y,2014.0,JAN,54.0,9.0,1.0,36904789,2,1,1


In [57]:
q5_online_treat_1  =  py_sql("SELECT customer_id2, \
                                     round(sum(net_purchase_amount), 2) as net_purchase_amount, \
                                     count(*) as Sales_Quantity, \
                                     Policy_Change\
                              FROM   q5_online_treat \
                              GROUP  BY customer_id2, Policy_Change;")

In [58]:
q5_online_cont_1 =  py_sql("SELECT customer_id2, \
                                     round(sum(net_purchase_amount), 2) as net_purchase_amount, \
                                     count(*) as Sales_Quantity, \
                                     Policy_Change \
                              FROM   q5_online_cont \
                              GROUP  BY customer_id2, Policy_Change;")

In [59]:
q5_online_cont_1['customer_id2'].nunique()

966

In [60]:
q5_online_treat_1['customer_id2'].nunique()

10374

In [61]:
#Finding out customer_ids that are present in both treatment and control 
intersection_5_online = py_sql("SELECT distinct customer_id2 \
                         FROM   q5_online_treat_1 \
                         WHERE customer_id2 in \
                               (                \
                               SELECT distinct customer_id2 \
                               FROM   q5_online_cont_1 \
                               );"       
                        )

In [62]:
# We should actually not get any common customer ids as this has already been taken care off in the query
# while generating q5_online_cont
intersection_5_online['customer_id2']

Series([], Name: customer_id2, dtype: object)

In [63]:
# just adding Treatment group to online_treat
q5_online_treat_2 = py_sql("SELECT *, \
                           1 as Treatment \
                          FROM q5_online_treat_1;")

In [64]:
# just adding Treatment group to online_cont
q5_online_cont_2 = py_sql("SELECT *, \
                           0 as Treatment \
                          FROM q5_online_cont_1;")

In [65]:
q5_online_cont_2.head(2)

,customer_id2,net_purchase_amount,Sales_Quantity,Policy_Change,Treatment
0,1000000001930,119.98,2.0,0,0
1,1000000001930,197.98,3.0,1,0


In [66]:
#Final data frame for online stores
q5_final_online = pd.concat([q5_online_treat_2, q5_online_cont_2])

In [67]:
q5_final_online.groupby(['Policy_Change'])['customer_id2'].count()

Policy_Change
0    11340
1    11340
Name: customer_id2, dtype: int64

In [68]:
q5_final_online[q5_final_online['net_purchase_amount']==0]
trans[trans['customer_id'] == 1000000015895]

,customer_id,purchase_date,transaction_id,store_number,net_purchase_amount,sku,sales_assoc_1,return,return_date,return_store,...,est_income_code,ethnic_code,homeowner_code,length_of_residence,child,year,month,month_index,summary,Brand_number
191011,1.000000e+12,19SEP2013:00:00:00,4784180.0,2.0,32.990002,18090142.0,0.0,1.0,13OCT2013:00:00:00,2.0,...,9.0,N,O,1.0,N,2013.0,SEP,50.0,2.0,1.0
295400,1.000000e+12,13OCT2013:00:00:00,4821541.0,2.0,0.000000,18090142.0,0.0,0.0,NaN,NaN,...,9.0,N,O,1.0,N,2013.0,OCT,51.0,2.0,1.0


In [69]:
q5_final_online.to_csv('M_Q5_Online.csv')

In [70]:
q5_final_online.count()

customer_id2           22680
net_purchase_amount    22680
Sales_Quantity         22680
Policy_Change          22680
Treatment              22680
dtype: int64

In [71]:
online_store = py_sql("SELECT * from trans where store_number in (2,6,10)")

In [72]:
online_store_1 = py_sql("SELECT customer_id, max(gender) as gender, max(age_band) as age_band, \
                              max(est_income_code) as est_income_code, \
                              max(ethnic_code) as ethnic_code, max(homeowner_code) as homeowner_code, \
                              max(length_of_residence) as length_of_residence, max(child) as child \
                         FROM online_store \
                         GROUP BY customer_id;")

In [73]:
online_store_1.head(2)

,customer_id,gender,age_band,est_income_code,ethnic_code,homeowner_code,length_of_residence,child
0,100273.0,M,11.0,9.0,N,O,14.0,N
1,100760.0,M,0.0,6.0,Z,O,5.0,Y


In [74]:
online_cust_att_final = py_sql("SELECT os.*, o.customer_id, o.gender, \
                                     o.age_band, o.est_income_code, o.ethnic_code, o.homeowner_code, \
                                     o.length_of_residence, o.child \
                              FROM   q5_final_online os   \
                              INNER JOIN online_store_1 o ON   \
                              os.customer_id2 = o.customer_id \
                              WHERE o.gender in ('M','F')  \
                              AND os.net_purchase_amount > 0;")

In [75]:
online_cust_att_final.count()

customer_id2           17721
net_purchase_amount    17721
Sales_Quantity         17721
Policy_Change          17721
Treatment              17721
customer_id            17721
gender                 17721
age_band               17625
est_income_code        17709
ethnic_code            17721
homeowner_code         17709
length_of_residence    17709
child                  17709
dtype: int64

In [76]:
online_cust_att_final.to_csv('M_q5_Online.csv')

### PHYSICAL STORE

In [77]:
q5_physical = py_sql("SELECT * from trans where store_number not in (2,6,10)")

In [78]:
# Data Cleansing
q5_physical['customer_id2'] = q5_physical.customer_id.astype(int).astype(str)
q5_physical['store_number2'] = q5_physical.store_number.astype(int).astype(str)
q5_physical['Brand_number2'] = q5_physical.Brand_number.astype(int).astype(str)

In [79]:
q5_physical.head(2)

,customer_id,purchase_date,transaction_id,store_number,net_purchase_amount,sku,sales_assoc_1,return,return_date,return_store,...,length_of_residence,child,year,month,month_index,summary,Brand_number,customer_id2,store_number2,Brand_number2
0,26888023.0,19SEP2013:00:00:00,2141863.0,1003.0,449.100010,17031584.0,NaN,1.0,19SEP2013:00:00:00,1003.0,...,2.0,N,2013.0,SEP,50.0,2.0,2.0,26888023,1003,2
1,38627803.0,08SEP2013:00:00:00,1142396.0,1003.0,99.989998,18541896.0,1044909.0,0.0,None,NaN,...,15.0,N,2013.0,SEP,50.0,21.0,2.0,38627803,1003,2


In [80]:
q5_physical['customer_id'].nunique()

1382782

In [81]:
q5_physical['store_number'].count()

2604817

In [82]:
q5_physical['Brand_number2'].unique()

array(['2', '3', '5', '8'], dtype=object)

In [83]:
q5_physical_treat = py_sql("SELECT *, \
                                (case when month_index > 50 then 1 else 0 end) as Policy_Change, \
                                (case when Brand_number2 in ('2','3','5') then 1 else 0 end ) as Treatment \
                      FROM q5_physical \
                      WHERE Brand_number2 in ('2','3','5') \
                      AND customer_id2 in \
                      (                    \
                         SELECT DISTINCT customer_id2 \
                         FROM q5_physical \
                         WHERE month_index > 50 \
                         AND Brand_number2 in ('2','3','5') \
                         AND customer_id2 in \
                         (                  \
                            SELECT DISTINCT customer_id2  \
                            FROM q5_physical \
                            where month_index <= 50 \
                            AND Brand_number2 in ('2','3','5')  \
                         ) \
                       )\
                       AND customer_id2 not in \
                       (                  \
                           SELECT DISTINCT customer_id2 \
                           FROM q5_physical \
                           WHERE month_index > 50 \
                           AND Brand_number2 in ('8') \
                           AND customer_id2 in \
                           (                  \
                               SELECT DISTINCT customer_id2  \
                               FROM q5_physical \
                               WHERE month_index <= 50 \
                               AND Brand_number2 in ('8')  \
                           )\
                        ) \
                        ;" )

In [84]:
q5_physical_cont = py_sql("SELECT *, \
                                (case when month_index > 50 then 1 else 0 end) as Policy_Change, \
                                (case when Brand_number2 in ('2','3','5') then 1 else 0 end ) as Treatment \
                      FROM q5_physical \
                      WHERE Brand_number2 in ('8') \
                      AND customer_id2 in \
                      (                    \
                         SELECT DISTINCT customer_id2 \
                         FROM q5_physical \
                         WHERE month_index > 50 \
                         AND Brand_number2 in ('8') \
                         AND customer_id2 in \
                         (                  \
                            SELECT DISTINCT customer_id2  \
                            FROM q5_physical \
                            where month_index <= 50 \
                            AND Brand_number2 in ('8')  \
                         ) \
                       )\
                       AND customer_id2 not in \
                       (                  \
                           SELECT DISTINCT customer_id2 \
                           FROM q5_physical \
                           WHERE month_index > 50 \
                           AND Brand_number2 in ('2','3','5') \
                           AND customer_id2 in \
                           (                  \
                               SELECT DISTINCT customer_id2  \
                               FROM q5_physical \
                               WHERE month_index <= 50 \
                               AND Brand_number2 in ('2','3','5')  \
                           )\
                        ) \
                        ;" )

In [85]:
#Data Cleansing
q5_physical_treat['Treatment'] = q5_physical_treat.Treatment.astype(int).astype(str)
q5_physical_treat['Policy_Change'] = q5_physical_treat.Policy_Change.astype(int).astype(str)

In [86]:
#Data Cleansing
q5_physical_cont['Treatment'] = q5_physical_cont.Treatment.astype(int).astype(str)
q5_physical_cont['Policy_Change'] = q5_physical_cont.Policy_Change.astype(int).astype(str)

In [87]:
#Checking if the sub table has same number of customer ids
q5_physical_treat['customer_id2'].nunique()

45757

In [88]:
#Checking if the sub table has same number of customer ids
q5_physical_cont['customer_id2'].nunique()

7534

In [89]:
#Creating new dataframe for Q6 without aggregation with both Treatment and Control
q5_without_aggregation_phy = pd.concat([q5_physical_treat, q5_physical_cont])

In [90]:
q5_without_aggregation_phy.head(5)

,customer_id,purchase_date,transaction_id,store_number,net_purchase_amount,sku,sales_assoc_1,return,return_date,return_store,...,year,month,month_index,summary,Brand_number,customer_id2,store_number2,Brand_number2,Policy_Change,Treatment
0,16295382.0,08DEC2013:00:00:00,1148219.0,1003.0,299.989990,18132555.0,455666.0,0.0,None,NaN,...,2013.0,DEC,53.0,21.0,2.0,16295382,1003,2,1,1
1,5199819.0,01NOV2013:00:00:00,1145683.0,1003.0,49.990002,19871623.0,1037042.0,0.0,None,NaN,...,2013.0,NOV,52.0,5.0,2.0,5199819,1003,2,1,1
2,21473844.0,10AUG2013:00:00:00,488320.0,1003.0,168.750000,18097600.0,455666.0,0.0,None,NaN,...,2013.0,AUG,49.0,13.0,2.0,21473844,1003,2,0,1
3,23827242.0,15DEC2013:00:00:00,1148817.0,1003.0,35.000000,19945317.0,1048782.0,0.0,None,NaN,...,2013.0,DEC,53.0,NaN,2.0,23827242,1003,2,1,1
4,37682307.0,31DEC2013:00:00:00,2144445.0,1003.0,199.990010,18500223.0,860485.0,0.0,None,NaN,...,2013.0,DEC,53.0,4.0,2.0,37682307,1003,2,1,1


In [91]:
q5_without_aggregation_phy.count()

customer_id            245419
purchase_date          245419
transaction_id         245419
store_number           245419
net_purchase_amount    245419
sku                    245419
sales_assoc_1          240560
return                 245419
return_date             35387
return_store            35387
time_to_return          35387
gender                 186592
age_band               200404
est_income_code        198929
ethnic_code            201772
homeowner_code         198929
length_of_residence    198929
child                  198929
year                   245419
month                  245419
month_index            245419
summary                233229
Brand_number           245419
customer_id2           245419
store_number2          245419
Brand_number2          245419
Policy_Change          245419
Treatment              245419
dtype: int64

In [92]:
q5_physical_treat_1  =  py_sql("SELECT customer_id2, \
                                     round(sum(net_purchase_amount), 2) as net_purchase_amount, \
                                     count(*) as Sales_Quantity, \
                                     Policy_Change\
                              FROM   q5_physical_treat \
                              GROUP  BY customer_id2, Policy_Change;")

In [93]:
q5_physical_cont_1 =  py_sql("SELECT customer_id2, \
                                     round(sum(net_purchase_amount), 2) as net_purchase_amount, \
                                     count(*) as Sales_Quantity, \
                                     Policy_Change \
                              FROM   q5_physical_cont \
                              GROUP  BY customer_id2, Policy_Change;")

In [94]:
#Rechecking if new dataset has same number of unique customer ids
q5_physical_cont_1['customer_id2'].nunique()

7534

In [95]:
#Rechecking if new dataset has same number of unique customer ids
q5_physical_treat_1['customer_id2'].nunique()

45757

In [96]:
#Finding out customer_ids that are present in both treatment and control 
intersection_5_physical = py_sql("SELECT distinct customer_id2 \
                         FROM   q5_physical_treat_1 \
                         WHERE customer_id2 in \
                               (                \
                               SELECT distinct customer_id2 \
                               FROM   q5_physical_cont_1 \
                               );"       
                        )

In [97]:
# We should actually not get any common customer ids as this has already been taken care off in the query
# while generating q5_online_cont
intersection_5_physical['customer_id2']

Series([], Name: customer_id2, dtype: object)

In [98]:
# just adding Treatment group to physical_treat_1
q5_physical_treat_2 = py_sql("SELECT *, \
                           1 as Treatment \
                          FROM q5_physical_treat_1;")

In [99]:
q5_physical.head(2)

,customer_id,purchase_date,transaction_id,store_number,net_purchase_amount,sku,sales_assoc_1,return,return_date,return_store,...,length_of_residence,child,year,month,month_index,summary,Brand_number,customer_id2,store_number2,Brand_number2
0,26888023.0,19SEP2013:00:00:00,2141863.0,1003.0,449.100010,17031584.0,NaN,1.0,19SEP2013:00:00:00,1003.0,...,2.0,N,2013.0,SEP,50.0,2.0,2.0,26888023,1003,2
1,38627803.0,08SEP2013:00:00:00,1142396.0,1003.0,99.989998,18541896.0,1044909.0,0.0,None,NaN,...,15.0,N,2013.0,SEP,50.0,21.0,2.0,38627803,1003,2


In [100]:
# just adding Treatment group to physical_cont_1
q5_physical_cont_2 = py_sql("SELECT *, \
                           0 as Treatment \
                          FROM q5_physical_cont_1;")

In [101]:
q5_physical_cont_2.head(2)

,customer_id2,net_purchase_amount,Sales_Quantity,Policy_Change,Treatment
0,1000000001048,22.48,2.0,0,0
1,1000000001048,22.48,2.0,1,0


In [102]:
#Final data frame for physical stores
q5_final_physical = pd.concat([q5_physical_treat_2, q5_physical_cont_2])

In [103]:
q5_final_physical.groupby(['Policy_Change'])['customer_id2'].count()

Policy_Change
0    53291
1    53291
Name: customer_id2, dtype: int64

In [104]:
q5_final_physical.count()

customer_id2           106582
net_purchase_amount    106582
Sales_Quantity         106582
Policy_Change          106582
Treatment              106582
dtype: int64

In [105]:
physical_store = py_sql("SELECT * FROM trans WHERE store_number not in (2,6,10)")

In [106]:
physical_store_1 = py_sql("SELECT customer_id, max(gender) as gender, max(age_band) as age_band, \
                              max(est_income_code) as est_income_code, \
                              max(ethnic_code) as ethnic_code, max(homeowner_code) as homeowner_code, \
                              max(length_of_residence) as length_of_residence, max(child) as child \
                         FROM physical_store \
                         GROUP BY customer_id;")

In [107]:
physical_store_1.head(2)

,customer_id,gender,age_band,est_income_code,ethnic_code,homeowner_code,length_of_residence,child
0,100014.0,F,12.0,6.0,H,O,15.0,N
1,100058.0,F,5.0,6.0,H,R,2.0,N


In [108]:
physical_cust_att_final = py_sql("SELECT ps.*, p.customer_id, p.gender, \
                                     p.age_band, p.est_income_code, p.ethnic_code, p.homeowner_code, \
                                     p.length_of_residence, p.child \
                              FROM   q5_final_physical ps   \
                              INNER JOIN physical_store_1 p ON   \
                              ps.customer_id2 = p.customer_id \
                              WHERE p.gender in ('M','F')  \
                              AND ps.net_purchase_amount > 0;")

In [109]:
physical_cust_att_final.count()

customer_id2           75555
net_purchase_amount    75555
Sales_Quantity         75555
Policy_Change          75555
Treatment              75555
customer_id            75555
gender                 75555
age_band               75269
est_income_code        75299
ethnic_code            75555
homeowner_code         75299
length_of_residence    75299
child                  75299
dtype: int64

In [110]:
physical_cust_att_final.to_csv('M_q5_Physical.csv')

### Q6. What is the impact of the policy change on customers return behavior?

#### Two Dataframes for Online q6_online_cust_at and For Physical q6_physical_cust_at

### ONLINE STORE

In [111]:
q6_online = py_sql("SELECT customer_id, net_purchase_amount, gender, age_band, est_income_code, ethnic_Code, homeowner_code, \
                      length_of_residence, child, Policy_Change, Treatment, return \
                      FROM q5_without_aggregation_onl \
                      WHERE gender in ('M','F');")

In [112]:
q6_online.head(2)

,customer_id,net_purchase_amount,gender,age_band,est_income_code,ethnic_code,homeowner_code,length_of_residence,child,Policy_Change,Treatment,return
0,36904789.0,18.75,F,0.0,6.0,S,O,1.0,Y,1,1,0.0
1,36210427.0,1549.99,M,0.0,5.0,S,O,10.0,Y,0,1,0.0


In [113]:
q6_online_nonan = q6_online[q6_online['child'].notnull()]

In [114]:
q6_online_nonan.to_csv('M_Q6_online.csv')

In [115]:
# Customers who  returned
q6_online.count()

customer_id            36831
net_purchase_amount    36831
gender                 36831
age_band               36624
est_income_code        36802
ethnic_code            36831
homeowner_code         36802
length_of_residence    36802
child                  36802
Policy_Change          36831
Treatment              36831
return                 36831
dtype: int64

### PHYSICAL

In [116]:
q6_physical = py_sql("SELECT customer_id, net_purchase_amount, gender, age_band, est_income_code, ethnic_Code, homeowner_code, \
                      length_of_residence, child, Policy_Change, Treatment, return \
                      FROM q5_without_aggregation_phy \
                      WHERE gender in ('M','F');")

In [117]:
q6_physical.head(2)

,customer_id,net_purchase_amount,gender,age_band,est_income_code,ethnic_code,homeowner_code,length_of_residence,child,Policy_Change,Treatment,return
0,16295382.0,299.989990,M,6.0,2.0,N,R,8.0,N,1,1,0.0
1,5199819.0,49.990002,F,0.0,2.0,S,R,7.0,N,1,1,0.0


In [118]:
q6_physical.head(2)

,customer_id,net_purchase_amount,gender,age_band,est_income_code,ethnic_code,homeowner_code,length_of_residence,child,Policy_Change,Treatment,return
0,16295382.0,299.989990,M,6.0,2.0,N,R,8.0,N,1,1,0.0
1,5199819.0,49.990002,F,0.0,2.0,S,R,7.0,N,1,1,0.0


In [119]:
q6_physical_nonan = q6_physical[q6_physical['child'].notnull()]

In [120]:
q6_physical_nonan.to_csv('M_Q6_physical.csv')

### CALCULATING PERCENTAGE RETURNS 

#### Calculating % Returns due to Policy Change

In [263]:
online_p = q5_without_aggregation_onl.loc[~(q5_without_aggregation_onl['gender'] != 
                                            q5_without_aggregation_onl['gender'])]


In [232]:
#online_p.count()

In [262]:
physical_p = q5_without_aggregation_phy.loc[~(q5_without_aggregation_phy['gender'] != 
                                              q5_without_aggregation_phy['gender'])]

In [254]:
#physical_p.count()

In [267]:
total_transactions_policy_change = (q5_without_aggregation_onl['return'].count() + 
                                   q5_without_aggregation_phy['return'].count())


In [268]:
total_rows_policy_change = online_p['return'].count() + physical_p['return'].count()
total_rows_policy_change

228277

In [271]:
total_return_policy_change = q6_physical[q6_physical['return'] > 0.0 ].count() + \
                             q6_online[q6_online['return'] > 0.0 ].count()
total_return_policy_change['return']

26499

In [158]:
total_transactions = trans['transaction_id'].count()
total_transactions

3218197

In [272]:
percentage_returns_policy_change = round(float(total_return_policy_change['return']) / 
                                         float(total_rows_policy_change)*100,2)
percentage_returns_policy_change

11.61

#### Percentage Returns due to Policy Change is 11.61%

In [148]:
# Calculating Original Return % from entire Dataset
total_return_entire_data = trans_p['return'].count()
total_return_entire_data

293964

In [252]:
# % Return from Entire Dataset
total_percentage_returns = round(float(total_return_entire_data) / float(total_transactions)*100,2)
total_percentage_returns

9.13

#### Percentage Returns from Entire Dataset is 9.13%

Total Percentage Increase in Returns = (11.61 - 9.13)/ 9.13 = 27.16%